In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [5]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [6]:
if __name__ == '__main__':

    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    links_list = []

    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

C:\Users\NISHIT\AppData\Local\Temp\ipykernel_7756\1605258723.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [7]:
amazon_df


,title,price,rating,reviews,availability
0,"Playstation Sony 4, 500GB Slim System [CUH-221...",,4.4 out of 5 stars,421 ratings,Not Available
1,PlayStation®5 Digital Edition (slim),,4.5 out of 5 stars,"10,685 ratings",Not Available
2,PS4 Slim 500GB Console + Controller,,4.4 out of 5 stars,"1,953 ratings",Not Available
3,PlayStation 4 Slim 1TB Console - Marvel's Spid...,,4.5 out of 5 stars,"2,832 ratings",Only 1 left in stock - order soon.
4,PlayStation 4 Pro 1TB Limited Edition Console ...,,4.1 out of 5 stars,675 ratings,Only 2 left in stock - order soon.
5,DualShock 4 Wireless Controller for PlayStatio...,,4.4 out of 5 stars,"123,026 ratings",Not Available
6,PlayStation 4 Slim 500GB Console - Uncharted 4...,,4.6 out of 5 stars,"6,557 ratings",Only 1 left in stock - order soon.
7,Sony Playstation Classic Console with 20 Plays...,,4.1 out of 5 stars,"2,191 ratings",Only 11 left in stock - order soon.
8,500GB PlayStation 4 Console - Limited Edition ...,,4.4 out of 5 stars,607 ratings,Only 1 left in stock - order soon.
9,PlayStation 4 Slim 1TB Limited Edition Console...,,4.6 out of 5 stars,819 ratings,Only 2 left in stock - order soon.
